In [2]:
import torchvision
from torchvision import transforms,datasets

import torch
import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.optim as optim
import seaborn as sns

import mlflow
import mlflow.pytorch

# Params


In [3]:
class Params(object):
    def __init__(self,batch_size,epochs,seed,log_interval):
        self.batch_size = batch_size
        self.epochs = epochs
        self.seed = seed
        self.log_interval = log_interval
        
args = Params(256,4,0,20)

# Dataset

In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.1307),(0.3081))]

)

train_set = datasets.MNIST('../data',train=True,download=True,
                          transform=transform)
test_set = datasets.MNIST('../data',train=False , download = True,
                        transform=transform)

trainloader = torch.utils.data.DataLoader(train_set,batch_size=args.batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(test_set,batch_size = args.batch_size,shuffle=False)


# Model

In [10]:
class Model(nn.Module):
    def __init__(self,nH=32):
        super(Model,self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(784,nH),
            nn.ReLU(),
            nn.Linear(nH,10)
            
        
        )
        
    def forward(self,x):
        x = x.view(x.size(0),-1)
        x= self.classifier(x)
        return x
    

In [11]:
def train(epoch):
    
    model.train()
    
    for batch_id,data in enumerate(trainloader):
        inputs,labels = trainloader
        
        opt.zero_grad()
        outputs = model(inputs)
        loss=loss_fn(outputs,labels)
        loss.backward()
        opt.step()
        
        if batch_id % args.log_interval==0:
            pos = epoch*len(trainloader)+batch_id
            mlflow.log_metric('train_loss',loss.data.item()/len(inputs)*1000)
            
            print('Train Epoch: {} [ {}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
                
                epoch,batch_id*len(inputs),len(trainloader.dataset),100. *batch_id/len(trainloader),loss.data.item()
            
            
            ))
                              
    

In [ ]:
def test(epoch):
    model.eval()
    test_loss=0
    correct=0
    confusion_matrix = np.zeros([10,10])
    
    with torch.no_grad():
        
        for inputs,labels in testloader:
            outputs = model(inputs)
            test_loss +=loss_fn(outputs,labels).data.item()
            pred = outputs.data.max(1)[1]
            correct += pred.eq(labels.data).sum().item()
            
            for x,y in zip(pred.numpy(),labels.numpy()):
                confusion_matrix[x][y] +=1